In [2]:
from pymongo import MongoClient
my_client = MongoClient('localhost', 27890, username='beihai', password='yaoduoxiang')
keyword_db = my_client['tops_keywords']

In [4]:
keyword_col = keyword_db['test']

In [5]:
sample = {"rank": 99, "keyword": "李嘉诚杀回内地市场", "score": "9.0万", "time": "2022-05-10 01:58:50"}

In [11]:
query = {'keyword':{'$eq': sample['keyword']}}

res = keyword_col.find(query,{})
res

In [12]:
list(res)

[]

In [14]:
import time
round(time.time()*1000)

1652427504740

In [1]:
d = {1:2}

In [3]:
res= d.get(2)
res

In [11]:
l1 = [{'k':1, 'id':0}, {'k':2, 'id':1}, {'k':3}]

In [12]:
l2 = [{'k':1},{'k':3}]

In [15]:
for item in l2:
    for xitem in l1:
        if (item['k'] == xitem['k']) and (xitem.get("id") is not None):
            item['_id'] = xitem.get('id')

In [16]:
l2

[{'k': 1, '_id': 0}, {'k': 3}]

In [18]:
# -*- coding: utf-8 -*-


from loguru import logger
import time
import random
from datetime import datetime
import pymongo
from pymongo.errors import DuplicateKeyError
from scrapy import Selector
from config import settings
from typing import Dict
import requests


############ other constants ################
# logger.debug(settings.user_agent_list)
user_agent_list = ["Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36",
                   "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36",
                   "Mozilla/5.0 (Windows NT 10.0) Gecko/20100101 Firefox/61.0",
                   "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36",
                   "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36",
                    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36",
                    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
                    "Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10.5; en-US; rv:1.9.2.15) Gecko/20110303 Firefox/3.6.15",
                   ]

headers = {
    "user-agent": random.choice(user_agent_list)
}

url = "https://top.baidu.com/board?tab=realtime"


################# database settings ###########
server_name = settings['server_name']
ip = settings[server_name].ip
mongo_settings = settings[server_name].dbs.mongo


my_client = pymongo.MongoClient(
    ip, mongo_settings.port, username=mongo_settings.usr, password=mongo_settings.pwd)
daily_db_name = settings['db_info']['mongo'].db_name
keyword_db_name = settings['db_info']['mongo'].db_name2
daily_db = my_client[daily_db_name]
keyword_db = my_client[keyword_db_name]


################ cralwer #####################
## 1. downloader ##

def downloader():
    """
    下载模块
    """
    resp = requests.get(url, headers=headers)
    resp.encoding = resp.apparent_encoding
    if resp.status_code == 200:
        return resp.text


def parser(html):
    """
    传入文本解析百度搜索的内容
    """
    selector = Selector(text=html)

    date = datetime.today().date().strftime("%Y-%m-%d")

    titles = selector.xpath(
        "//a[@class='title_dIF3B ']/div[@class='c-single-text-ellipsis']/text()").extract()
    scores = selector.xpath(
        "//div[@class='trend_2RttY hide-icon']/div[@class='hot-index_1Bl1a']/text()").extract()
    # urls = selector.xpath("")
    data = []
    if len(titles) == len(scores):
        ranks = [i for i in range(0, len(scores))]

        for i, item in enumerate(titles):
            d = {}
            d['keyword'] = titles[i]
            d['rank'] = ranks[i]
            d['score'] = scores[i]
            d['time'] = date
            data.append(d)

    day_data = {}
    day_data['_id'] = date
    day_data['crawltime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    day_data['data'] = data
    day_data['rank_name'] = 'baidu'
    return day_data


def saver(day_data: Dict):
    baidu_col = daily_db['baidu']
    date = datetime.today().date().strftime("%Y-%m-%d")
    try:
        res = baidu_col.insert_one(day_data)

        logger.info(f'SAVE 【{date}-baidu】data succ')
    except Exception as e:
        if type(e) == DuplicateKeyError:
            logger.warning(f"already loged {date} data, update now")

            # 更新数据
            old_data = list(baidu_col.find_one({'_id':{'$eq': day_data['_id']}},{})) # 因为是duplicate key了数据必然存在
            logger.debug(old_data)
            old_news_list = old_data['data']
            new_news_list = day_data['data']
            for new_news in new_news_list:
                for old_news in old_news_list:
                    # 有tag则更新tag
                    if (new_news['keyword'] == old_news['keyword']) and (old_data.get('tag') is not None):
                        new_news['tag'] = old_data.get('tag')
                    if(new_news['keyword'] == old_news['keyword']) and (old_data.get('url') is not None):
                        new_news['url'] = old_news.get('url')
            baidu_col.update_one({'_id': day_data['_id']}, {
                                 '$set': day_data})
        else:
            logger.error(e)


def baidu_top_crawler():
    download_res = downloader()
    parse_res = parser(download_res)
    saver(parse_res)


if __name__ == "__main__":
    baidu_top_crawler()


2022-06-01 00:34:55.977 | WARNING  | __main__:saver:104 - already loged 2022-06-01 data, update now
2022-06-01 00:34:55.980 | DEBUG    | __main__:saver:108 - ['_id', 'crawltime', 'data', 'rank_name']


TypeError: list indices must be integers or slices, not str

In [21]:
baidu_col = daily_db['baidu']
res = baidu_col.find({'_id': '2022-06-01'})
res

In [13]:
res = downloader()
selector = Selector(text=res)

date = datetime.today().date().strftime("%Y-%m-%d")

titles = selector.xpath("//a[@class='title_dIF3B ']/div[@class='c-single-text-ellipsis']/text()").extract()
scores = selector.xpath("//div[@class='trend_2RttY hide-icon']/div[@class='hot-index_1Bl1a']/text()").extract()
# urls = selector.xpath("")
data = []
if len(titles) == len(scores):
    ranks = [i for i in range(0,len(scores))]
    
    for i,item in enumerate(titles):
        d = {}
        d['keyword'] = titles[i]
        d['rank'] = ranks[i]
        d['score'] = scores[i]
        d['time'] = date
        data.append(d)

day_data = {}
day_data['_id'] = date
day_data['crawltime'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
day_data['data'] = data

In [14]:
day_data

{'_id': '2022-05-31',
 'crawltime': '2022-05-31 21:54:38',
 'data': [{'keyword': '  六方面33项稳经济措施 ',
   'rank': 0,
   'score': ' 4998604 ',
   'time': '2022-05-31'},
  {'keyword': '  四平新通报删除2次以上未核酸拘留 ',
   'rank': 1,
   'score': ' 4953017 ',
   'time': '2022-05-31'},
  {'keyword': '  女子称因生理期上厕所次数多被裁 ',
   'rank': 2,
   'score': ' 4830124 ',
   'time': '2022-05-31'},
  {'keyword': '  把美的种子播撒在孩子心中 ',
   'rank': 3,
   'score': ' 4710511 ',
   'time': '2022-05-31'},
  {'keyword': '  江苏省委原副书记张敬华被双开 ',
   'rank': 4,
   'score': ' 4640689 ',
   'time': '2022-05-31'},
  {'keyword': '  被封男子吃绿化带发现84种草药 ',
   'rank': 5,
   'score': ' 4503868 ',
   'time': '2022-05-31'},
  {'keyword': '  茅台镇酒厂起火 上百吨酱酒流入河 ',
   'rank': 6,
   'score': ' 4414251 ',
   'time': '2022-05-31'},
  {'keyword': '  中小学教师性别严重失衡 ',
   'rank': 7,
   'score': ' 4369088 ',
   'time': '2022-05-31'},
  {'keyword': '  干部嫖娼被罚未获处分？单位回应 ',
   'rank': 8,
   'score': ' 4273261 ',
   'time': '2022-05-31'},
  {'keyword': '  北京1例确诊此前17次核酸阴性 '

In [1]:
from pymongo import MongoClient
import pymongo
from config import settings
from loguru import logger

In [2]:
################# database settings ###########
server_name = settings['server_name']
ip = settings[server_name].ip
mongo_settings = settings[server_name].dbs.mongo


my_client = pymongo.MongoClient(
    ip, mongo_settings.port, username=mongo_settings.usr, password=mongo_settings.pwd)
daily_db_name = settings['db_info']['mongo'].db_name
keyword_db_name = settings['db_info']['mongo'].db_name2
daily_db = my_client[daily_db_name]
keyword_db = my_client[keyword_db_name]
################ cralwer #####################
daily_db_names = daily_db.list_collection_names()

In [7]:
# 读取没有url的daily数据
def query_keyword_url(keyword:str, col_name:str):
    query = {'keyword': {'$eq': keyword}}
    projection = {'url': 1}
    try:
        url = list(keyword_db[col_name].find(query, projection))[0].get('url')
        if url is not None:
            return url
    except Exception as e:
        logger.error(f"{keyword} 's url not found, error is {e}")

def update_daily_db_url(col_name: str):
    query = {'data': {'$elemMatch': {'url': {'$exists': False}}}}
    none_url_data = list(daily_db[col_name].find(query))
    num = len(none_url_data)
    logger.info(f"got {num} keywords")
    
    for item in none_url_data:
        for each_tops in item['data']:
            url = query_keyword_url(each_tops['keyword'], col_name)
            each_tops['url'] = url
    
        update_res = daily_db[col_name].update_one(item, {'$set' : {'data': item['data']}})
        logger.debug(f"update {item['_id']} result is {update_res.raw_result}")

def update_all_daily_db_url():
    for col_name in daily_db_names:
        update_daily_db_url(col_name) 

In [9]:
for col_name in daily_db_names:
    update_daily_db_url(col_name) 

2022-06-04 02:42:05.304 | INFO     | __main__:update_daily_db_url:16 - got 6 keywords
2022-06-04 02:42:05.428 | DEBUG    | __main__:update_daily_db_url:24 - update 2022-05-29 result is {'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}
2022-06-04 02:42:05.550 | DEBUG    | __main__:update_daily_db_url:24 - update 2022-05-30 result is {'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}
2022-06-04 02:42:05.672 | DEBUG    | __main__:update_daily_db_url:24 - update 2022-05-31 result is {'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}
2022-06-04 02:42:05.793 | DEBUG    | __main__:update_daily_db_url:24 - update 2022-06-01 result is {'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}
2022-06-04 02:42:05.914 | DEBUG    | __main__:update_daily_db_url:24 - update 2022-06-02 result is {'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}
2022-06-04 02:42:06.030 | DEBUG    | __main__:update_daily_db_url:24 - update 2022-06-03 result is {'n': 0, '

In [14]:
res = read_none_url_daily('ZAKER')

2022-06-04 01:59:23.964 | INFO     | __main__:read_none_url_daily:6 - got 22 keywords


In [16]:
res[0]['data']

[{'rank': 0,
  'keyword': '全国大中型灌区春灌进度过六成',
  'score': '水利部 ‧ 央视新闻',
  'time': '2022-05-13',
  'tag': '社会',
  'url': 'http://baijiahao.baidu.com/s?id=1732540563162400712&wfr=spider&for=pc'},
 {'rank': 1,
  'keyword': '部署防灾减灾工作 习近平提出树牢安全发展理念',
  'score': '习之 ‧ 人民网-中国共产党新闻网',
  'time': '2022-05-13',
  'tag': '社会',
  'url': 'http://news.cri.cn/n/20220512/7211c3ec-d9df-78d6-e32c-ef5aca88488b.html'},
 {'rank': 2,
  'keyword': '被迫消失八年，是谁“害”了黄海波？',
  'score': '黄海波 ‧ 往事叉烧',
  'time': '2022-05-13',
  'tag': '社会',
  'url': 'https://www.163.com/dy/article/H8CR7DMJ05534I3H.html'},
 {'rank': 3,
  'keyword': '合作是实现经济复苏唯一出路（观象台）',
  'score': '经济全球化 ‧ 海外网',
  'time': '2022-05-13',
  'tag': '其他',
  'url': 'https://baijiahao.baidu.com/s?id=1720699750461691017&wfr=spider&for=pc'},
 {'rank': 4,
  'keyword': '部分地区默认关闭接听国际及港澳台电话，官方回应：属实',
  'score': '中国移动 ‧ 蓝鲸财经',
  'time': '2022-05-13',
  'tag': '社会',
  'url': 'http://news.cnr.cn/native/gd/20220516/t20220516_525827950.shtml'},
 {'rank': 5,
  'keyword': '中方

In [9]:
# threadpool
import threadpool
def t(n,m):
    print(n+m)
    return m+n



In [12]:
pool=threadpool.ThreadPool(5)
# logger.info(f'根据读取的col数量生成{thread_num}个线程')
list_of_n = [(1,2),(3,2),(3,3),(3,4),(3,5)]
tasks=threadpool.makeRequests(t, list_of_n)
[pool.putRequest(task) for task in tasks]
pool.wait()

Traceback (most recent call last):
  File "/srv/BigData/chentianxuan/anaconda3/envs/spiders/lib/python3.8/site-packages/threadpool.py", line 158, in run
    result = request.callable(*request.args, **request.kwds)
TypeError: t() argument after ** must be a mapping, not int
Traceback (most recent call last):
  File "/srv/BigData/chentianxuan/anaconda3/envs/spiders/lib/python3.8/site-packages/threadpool.py", line 158, in run
    result = request.callable(*request.args, **request.kwds)
TypeError: t() argument after ** must be a mapping, not int
Traceback (most recent call last):
  File "/srv/BigData/chentianxuan/anaconda3/envs/spiders/lib/python3.8/site-packages/threadpool.py", line 158, in run
    result = request.callable(*request.args, **request.kwds)
TypeError: t() argument after ** must be a mapping, not int
Traceback (most recent call last):
  File "/srv/BigData/chentianxuan/anaconda3/envs/spiders/lib/python3.8/site-packages/threadpool.py", line 158, in run
    result = request.call